In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
import os
from langchain_nomic import NomicEmbeddings
from dotenv import load_dotenv
# from openai.error import RateLimitError
from langchain_openai import ChatOpenAI
import networkx as nx
import fitz
from typing import List
import numpy as np
from knowledge_graph import KnowledgeGraph
from query_engine import QueryEngine
from langchain.document_loaders import  PyPDFLoader
from concurrent.futures import ThreadPoolExecutor, as_completed
import tqdm as tqdm
from langchain_core.prompts import PromptTemplate
# from typing import 
from langchain_core.pydantic_v1 import BaseModel, Field
import spacy
from spacy.cli import download

class Concepts(BaseModel):
    concepts_list: List[str] = Field(description="List of concepts")


load_dotenv()  # Load environment variables from .env file

api_key = os.getenv("NOMIC_API_KEY")

path = "/home/name-1/AI-Agent/RAG_Project/RAG_Project/data/Understanding_Climate_Change.pdf"
loader = PyPDFLoader(path)
documents = loader.load()
documents = documents[:10]

# DocumentProcessor's process_documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
embeddings =  NomicEmbeddings(model="nomic-embed-text-v1.5",)
splits = text_splitter.split_documents(documents)
vector_store = FAISS.from_documents(splits, embeddings)
graph = nx.Graph()

knowledge_graph = KnowledgeGraph()
llm = ChatOpenAI(
                model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
                base_url="http://10.2.125.37:1234/v1",
                api_key="lm-studio"
            )

In [10]:
splits = text_splitter.split_documents(documents)
vector_store = FAISS.from_documents(splits, embeddings)

In [11]:
for i, split in enumerate(splits):
    graph.add_node(i, content=split.page_content)

In [12]:
texts = [split.page_content for split in splits]
create_embedding = embeddings.embed_documents(texts)

In [13]:
concept_cache = {}
def _load_spacy_model():
    """
    Loads the spaCy NLP model, downloading it if necessary.
    
    Args:
    - None
    
    Returns:
    - spacy.Language: An instance of a spaCy NLP model.
    """
    try:
        return spacy.load("en_core_web_sm")
    except OSError:
        print("Downloading spaCy model...")
        download("en_core_web_sm")
        return spacy.load("en_core_web_sm")
nlp = _load_spacy_model()

In [14]:
def _extract_concepts_and_entities(content, llm):
    """
    Extracts concepts and named entities from the content using spaCy and a large language model.
    
    Args:
    - content (str): The content from which to extract concepts and entities.
    - llm: An instance of a large language model.
    
    Returns:
    - list: A list of extracted concepts and entities.
    """
    if content in concept_cache:
        return concept_cache[content]
    
    # Extract named entities using spaCy
    doc = nlp(content)
    named_entities = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE", "WORK_OF_ART"]]
    
    # Extract general concepts using LLM
    concept_extraction_prompt = PromptTemplate(
        input_variables=["text"],
        template="Extract key concepts (excluding named entities) from the following text:\n\n{text}\n\nKey concepts:"
    )
    concept_chain = concept_extraction_prompt | llm.with_structured_output(Concepts)
    general_concepts = concept_chain.invoke({"text": content}).concepts_list
    
    # Combine named entities and general concepts
    all_concepts = list(set(named_entities + general_concepts))
    
    concept_cache[content] = all_concepts
    return all_concepts

In [ ]:
with ThreadPoolExecutor() as executor:
    future_to_node = {executor.submit(_extract_concepts_and_entities, split.page_content, llm): i 
                        for i, split in enumerate(splits)}
    
    for future in tqdm(as_completed(future_to_node), total=len(splits), desc="Extracting concepts and entities"):
        node = future_to_node[future]
        concepts = future.result()
        graph.nodes[node]['concepts'] = concepts